In [1]:
HTML(read(open("style.html"), String))

HTML{String}("<link href='https://fonts.googleapis.com/css?family=Alegreya+Sans:100,300,400,500,700,800,900,100italic,300italic,400italic,500italic,700italic,800italic,900italic' rel='stylesheet' type='text/css'>\r\n<link href='https://fonts.googleapis.com/css?family=Arvo:400,700,400italic' rel='stylesheet' type='text/css'>\r\n<link href='https://fonts.googleapis.com/css?family=PT+Mono' rel='stylesheet' type='text/css'>\r\n<link href='https://fonts.googleapis.com/css?family=Shadows+Into+Light' rel='stylesheet' type='text/css'>\r\n<link href='https://fonts.googleapis.com/css?family=Philosopher:400,700,400italic,700italic' rel='stylesheet' type='text/css'>\r\n\r\n<style>\r\n\r\n@font-face {\r\n    font-family: \"Computer Modern\";\r\n    src: url('https://mirrors.ctan.org/fonts/cm-unicode/fonts/otf/cmunss.otf');\r\n}\r\n\r\n.container { width: 100% }\r\n\r\n/* Formatting for header cells */\r\n.text_cell_render h1 {\r\n    font-family: 'Philosopher', sans-serif;\r\n    font-weight: 400;\r\n    font-size: 2.2em;\r\n    line-height: 100%;\r\n    color: rgb(0, 80, 120);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n.text_cell_render h2 {\r\n    font-family: 'Philosopher', serif;\r\n    font-weight: 400;\r\n    font-size: 1.9em;\r\n    line-height: 100%;\r\n    color: rgb(200,100,0);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n\r\n.text_cell_render h3 {\r\n    font-family: 'Philosopher', serif;\r\n    margin-top:12px;\r\n    margin-bottom: 3px;\r\n    font-style: italic;\r\n    color: rgb(94,127,192);\r\n}\r\n\r\n.text_cell_render h4 {\r\n    font-family: 'Philosopher', serif;\r\n}\r\n\r\n.text_cell_render h5 {\r\n    font-family: 'Alegreya Sans', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 16pt;\r\n    color: grey;\r\n    font-style: italic;\r\n    margin-bottom: .1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\r\n\r\n.text_cell_render h6 {\r\n    font-family: 'PT Mono', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 10pt;\r\n    color: grey;\r\n    margin-bottom: 1px;\r\n    margin-top: 1px;\r\n}\r\n\r\n.text_cell_render em {\r\n    font-family: 'Philosopher', sans-serif;\r\n    color:        blue;\r\n    background-color: rgb(255,220,180);\r\n    font-size:    110%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   100;\r\n}\r\n\r\n.text_cell_render b {\r\n    color:            rgb(255,195,195);\r\n    background-color: rgb(0,0,0);\r\n    font-size:    110%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   650;\r\n}\r\n\r\n.text_cell_render tt {\r\n    font-size:    120%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   150;\r\n}\r\n\r\n.Codemirror {\r\n    font-family: \"PT Mono\", sans-serif;\r\n    font-size: 100%;\r\n}\r\n\r\n</style>")

In [2]:
import Chess

# Bewertung des Spielfeldes

Um den aktuellen Spielstand eines Schachspiels, bzw. die Stärke zukünftiger Züge bestimmen zu können, wird zunächst die Wertigkeit jeder Spielfigur bestimmt. Diese Wertigkeiten werden anschließend von den Funktionen des Notebooks [`Evaluation`](3.1%20-%20Evaluation.ipynb) verwendet, um die berechneten Züge zu bewerten. Die verschiedenen implementierten Algorithmen nutzen diese Bewertung, um die bestmöglichen Züge wiederzugeben.

Die `Piece Values` und die `Piece Square Values` stammen von [Tomasz Michniewski](https://www.chessprogramming.org/Tomasz_Michniewski) [9], einem polnischen Informatiker und wurden diesem [Chess-Wiki](https://www.chessprogramming.org/Simplified_Evaluation_Function) [10] entnommen. Sie wurden in dem polnischen Schachprogrammierung-Forum "progszach" zu Diskussionszwecken veröffentlicht.

## Piece Value

Wir beginnen mit der Definition der Spielfigur-Werte oder auch Piece Values. Die Bewertung ist in Centi-Pawns angegeben. Ein Centi-Pawn ist eine Maßeinheit zur Berechnung des Vorteils im Schach und wird hauptsächlich von Spielern und Anwendungen im Computer-Schach zur Positionsevaluierung verwendet. Ein Centi-Pawn besitzt dabei die Wertigkeit von 1/100 eines Bauern.

Der tatsächliche Wert einer Figur ist abhängig von dem Nutzen sowie der Wichtigkeit der Figur. Es werden somit folgende Wertigkeiten festgehalten:

- Pawn: 100
- Knight: 320
- Bishop: 330
- Rook: 500
- Queen: 900
- King: 0

Diese Werte gelten für beide Seiten. 

Da der König als Figur nicht schlagbar ist, und jede Spielpartei genau einen König besitzt, ist dessen Wert zu vernachlässigen und kann theoretisch mit jeder Wertigkeit belegt werden. Um die Ausgabe und den Werte-Rahmen so simpel wie möglich zu gestalten, haben wir uns für den Wert 0 entschieden.

Die Definition als Konstante wurde gewählt, damit der Zugriff auf diese Werte vom Just-in-time Compiler optimiert werden kann.

In [3]:
if !@isdefined(PIECE_VALUE_PAWN)
    const PIECE_VALUE_PAWN::Int32 = 100
end

100

In [4]:
if !@isdefined(PIECE_VALUE_KNIGHT)
    const PIECE_VALUE_KNIGHT::Int32 = 320
end

320

In [5]:
if !@isdefined(PIECE_VALUE_BISHOP)
    const PIECE_VALUE_BISHOP::Int32 = 330
end

330

In [6]:
if !@isdefined(PIECE_VALUE_ROOK)
    const PIECE_VALUE_ROOK::Int32 = 500
end

500

In [7]:
if !@isdefined(PIECE_VALUE_QUEEN)
    const PIECE_VALUE_QUEEN::Int32 = 900
end

900

In [8]:
if !@isdefined(PIECE_VALUE_KING)
    const PIECE_VALUE_KING::Int32 = 0
end

0

Die einzelnen Konstanten werden anschließend in einem Array zusammengefasst.

In [9]:
if !@isdefined(PIECE_VALUES)
    const PIECE_VALUES::Array{Int32} = [
        PIECE_VALUE_PAWN, 
        PIECE_VALUE_KNIGHT, 
        PIECE_VALUE_BISHOP,
        PIECE_VALUE_ROOK,
        PIECE_VALUE_QUEEN,
        PIECE_VALUE_KING
    ]
end

6-element Vector{Int32}:
 100
 320
 330
 500
 900
   0

***

## Piece Square Tables

Spielfiguren besitzen je nach Position eine unterschiedliche Wertigkeit. Beispielweise ist von deutlichem Vorteil, einen Bauer in den oberen Bereich des Spielfeldes zu bewegen, um eine Promotion auf eine höherwertige Figur zu vollziehen.

Um die Positionsstärke einer Figur auf einem gegebenen Spielfeld schnell und effektiv abfragen zu können, definieren wir für jeden Figurtyp eine Gewicht-Matrix ([Piece Square Table](https://www.chessprogramming.org/Simplified_Evaluation_Function)).
<br>

Um die schwarzen `Piece Square Tables` korrekt zu verwenden, müssen die weißen Werte wieder konvertiert werden. Dazu wird zuerst die Matrix an der x-Achse gespiegelt. Dies symbolisiert die Spielrichtung von Schwarz.
<br>

In [10]:
if !@isdefined(PieceSquareTable)
    const PieceSquareTable = Matrix{Int32}
end

Matrix{Int32} (alias for Array{Int32, 2})

In [11]:
if !@isdefined(PIECE_SQUARE_TABLE_WP)
    const PIECE_SQUARE_TABLE_WP::PieceSquareTable = [ 
        [0    0   0   0   0   0   0   0] 
        [50  50  50  50  50  50  50  50] 
        [10  10  20  30  30  20  10  10] 
        [5    5  10  25  25  10   5   5] 
        [0    0   0  20  20   0   0   0] 
        [5   -5 -10   0   0 -10  -5   5] 
        [5   10  10 -20 -20  10  10   5] 
        [0    0   0   0   0   0   0   0]  
    ]
end

8×8 Matrix{Int64}:
  0   0    0    0    0    0   0   0
 50  50   50   50   50   50  50  50
 10  10   20   30   30   20  10  10
  5   5   10   25   25   10   5   5
  0   0    0   20   20    0   0   0
  5  -5  -10    0    0  -10  -5   5
  5  10   10  -20  -20   10  10   5
  0   0    0    0    0    0   0   0

In [12]:
if !@isdefined(PIECE_SQUARE_TABLE_BP)
    const PIECE_SQUARE_TABLE_BP::PieceSquareTable = reverse(PIECE_SQUARE_TABLE_WP, 
    dims = 1)
end

8×8 Matrix{Int32}:
  0   0    0    0    0    0   0   0
  5  10   10  -20  -20   10  10   5
  5  -5  -10    0    0  -10  -5   5
  0   0    0   20   20    0   0   0
  5   5   10   25   25   10   5   5
 10  10   20   30   30   20  10  10
 50  50   50   50   50   50  50  50
  0   0    0    0    0    0   0   0

In [13]:
if !@isdefined(PIECE_SQUARE_TABLE_WN)
    const PIECE_SQUARE_TABLE_WN::PieceSquareTable = [
        [-50 -40 -30 -30 -30 -30 -40 -50] 
        [-40 -20   0   0   0   0 -20 -40] 
        [-30   0  10  15  15  10   0 -30] 
        [-30   5  15  20  20  15   5 -30] 
        [-30   0  15  20  20  15   0 -30] 
        [-30   5  10  15  15  10   5 -30] 
        [-40 -20   0   5   5   0 -20 -40] 
        [-50 -40 -30 -30 -30 -30 -40 -50]    
    ]
end

8×8 Matrix{Int64}:
 -50  -40  -30  -30  -30  -30  -40  -50
 -40  -20    0    0    0    0  -20  -40
 -30    0   10   15   15   10    0  -30
 -30    5   15   20   20   15    5  -30
 -30    0   15   20   20   15    0  -30
 -30    5   10   15   15   10    5  -30
 -40  -20    0    5    5    0  -20  -40
 -50  -40  -30  -30  -30  -30  -40  -50

In [14]:
if !@isdefined(PIECE_SQUARE_TABLE_BN)
    const PIECE_SQUARE_TABLE_BN::PieceSquareTable = reverse(PIECE_SQUARE_TABLE_WN, 
    dims = 1)
end

8×8 Matrix{Int32}:
 -50  -40  -30  -30  -30  -30  -40  -50
 -40  -20    0    5    5    0  -20  -40
 -30    5   10   15   15   10    5  -30
 -30    0   15   20   20   15    0  -30
 -30    5   15   20   20   15    5  -30
 -30    0   10   15   15   10    0  -30
 -40  -20    0    0    0    0  -20  -40
 -50  -40  -30  -30  -30  -30  -40  -50

In [15]:
if !@isdefined(PIECE_SQUARE_TABLE_WB)
    const PIECE_SQUARE_TABLE_WB::PieceSquareTable = [
        [-20 -10 -10 -10 -10 -10 -10 -20] 
        [-10   0   0   0   0   0   0 -10] 
        [-10   0   5  10  10   5   0 -10] 
        [-10   5   5  10  10   5   5 -10] 
        [-10   0  10  10  10  10   0 -10] 
        [-10  10  10  10  10  10  10 -10] 
        [-10   5   0   0   0   0   5 -10] 
        [-20 -10 -10 -10 -10 -10 -10 -20]                                    
    ]
end

8×8 Matrix{Int64}:
 -20  -10  -10  -10  -10  -10  -10  -20
 -10    0    0    0    0    0    0  -10
 -10    0    5   10   10    5    0  -10
 -10    5    5   10   10    5    5  -10
 -10    0   10   10   10   10    0  -10
 -10   10   10   10   10   10   10  -10
 -10    5    0    0    0    0    5  -10
 -20  -10  -10  -10  -10  -10  -10  -20

In [16]:
if !@isdefined(PIECE_SQUARE_TABLE_BB)
    const PIECE_SQUARE_TABLE_BB::PieceSquareTable = reverse(PIECE_SQUARE_TABLE_WB, 
    dims = 1)
end

8×8 Matrix{Int32}:
 -20  -10  -10  -10  -10  -10  -10  -20
 -10    5    0    0    0    0    5  -10
 -10   10   10   10   10   10   10  -10
 -10    0   10   10   10   10    0  -10
 -10    5    5   10   10    5    5  -10
 -10    0    5   10   10    5    0  -10
 -10    0    0    0    0    0    0  -10
 -20  -10  -10  -10  -10  -10  -10  -20

In [17]:
if !@isdefined(PIECE_SQUARE_TABLE_WR)
    const PIECE_SQUARE_TABLE_WR::PieceSquareTable = [
        [  0   0   0   0   0   0   0   0] 
        [  5  10  10  10  10  10  10   5] 
        [ -5   0   0   0   0   0   0  -5] 
        [ -5   0   0   0   0   0   0  -5] 
        [ -5   0   0   0   0   0   0  -5] 
        [ -5   0   0   0   0   0   0  -5] 
        [ -5   0   0   0   0   0   0  -5] 
        [  0   0   0   5   5   0   0   0]                             
    ]
end

8×8 Matrix{Int64}:
  0   0   0   0   0   0   0   0
  5  10  10  10  10  10  10   5
 -5   0   0   0   0   0   0  -5
 -5   0   0   0   0   0   0  -5
 -5   0   0   0   0   0   0  -5
 -5   0   0   0   0   0   0  -5
 -5   0   0   0   0   0   0  -5
  0   0   0   5   5   0   0   0

In [18]:
if !@isdefined(PIECE_SQUARE_TABLE_BR)
    const PIECE_SQUARE_TABLE_BR::PieceSquareTable = reverse(PIECE_SQUARE_TABLE_WR, 
    dims = 1)
end

8×8 Matrix{Int32}:
  0   0   0   5   5   0   0   0
 -5   0   0   0   0   0   0  -5
 -5   0   0   0   0   0   0  -5
 -5   0   0   0   0   0   0  -5
 -5   0   0   0   0   0   0  -5
 -5   0   0   0   0   0   0  -5
  5  10  10  10  10  10  10   5
  0   0   0   0   0   0   0   0

In [19]:
if !@isdefined(PIECE_SQUARE_TABLE_WQ)
    const PIECE_SQUARE_TABLE_WQ::PieceSquareTable = [ 
        [-20 -10 -10  -5  -5 -10 -10 -20] 
        [-10   0   0   0   0   0   0 -10] 
        [-10   0   5   5   5   5   0 -10] 
        [ -5   0   5   5   5   5   0  -5] 
        [  0   0   5   5   5   5   0  -5] 
        [-10   5   5   5   5   5   0 -10] 
        [-10   0   5   0   0   0   0 -10] 
        [-20 -10 -10  -5  -5 -10 -10 -20]                              
    ]
end

8×8 Matrix{Int64}:
 -20  -10  -10  -5  -5  -10  -10  -20
 -10    0    0   0   0    0    0  -10
 -10    0    5   5   5    5    0  -10
  -5    0    5   5   5    5    0   -5
   0    0    5   5   5    5    0   -5
 -10    5    5   5   5    5    0  -10
 -10    0    5   0   0    0    0  -10
 -20  -10  -10  -5  -5  -10  -10  -20

In [20]:
if !@isdefined(PIECE_SQUARE_TABLE_BQ)
    const PIECE_SQUARE_TABLE_BQ::PieceSquareTable = reverse(PIECE_SQUARE_TABLE_WQ, 
    dims = 1)
end

8×8 Matrix{Int32}:
 -20  -10  -10  -5  -5  -10  -10  -20
 -10    0    5   0   0    0    0  -10
 -10    5    5   5   5    5    0  -10
   0    0    5   5   5    5    0   -5
  -5    0    5   5   5    5    0   -5
 -10    0    5   5   5    5    0  -10
 -10    0    0   0   0    0    0  -10
 -20  -10  -10  -5  -5  -10  -10  -20

In [21]:
if !@isdefined(PIECE_SQUARE_TABLE_WK)
    const PIECE_SQUARE_TABLE_WK::PieceSquareTable = [
        [-30 -40 -40 -50 -50 -40 -40 -30] 
        [-30 -40 -40 -50 -50 -40 -40 -30] 
        [-30 -40 -40 -50 -50 -40 -40 -30] 
        [-30 -40 -40 -50 -50 -40 -40 -30] 
        [-20 -30 -30 -40 -40 -30 -30 -20] 
        [-10 -20 -20 -20 -20 -20 -20 -10] 
        [ 20  20   0   0   0   0  20  20] 
        [ 20  30  10   0   0  10  30  20]
    ]
end

8×8 Matrix{Int64}:
 -30  -40  -40  -50  -50  -40  -40  -30
 -30  -40  -40  -50  -50  -40  -40  -30
 -30  -40  -40  -50  -50  -40  -40  -30
 -30  -40  -40  -50  -50  -40  -40  -30
 -20  -30  -30  -40  -40  -30  -30  -20
 -10  -20  -20  -20  -20  -20  -20  -10
  20   20    0    0    0    0   20   20
  20   30   10    0    0   10   30   20

In [22]:
if !@isdefined(PIECE_SQUARE_TABLE_BK)
    const PIECE_SQUARE_TABLE_BK::PieceSquareTable = reverse(PIECE_SQUARE_TABLE_WK, 
    dims = 1)
end

8×8 Matrix{Int32}:
  20   30   10    0    0   10   30   20
  20   20    0    0    0    0   20   20
 -10  -20  -20  -20  -20  -20  -20  -10
 -20  -30  -30  -40  -40  -30  -30  -20
 -30  -40  -40  -50  -50  -40  -40  -30
 -30  -40  -40  -50  -50  -40  -40  -30
 -30  -40  -40  -50  -50  -40  -40  -30
 -30  -40  -40  -50  -50  -40  -40  -30

In [23]:
if !@isdefined(PIECE_SQUARE_TABLE_WK_EG)
    const PIECE_SQUARE_TABLE_WK_EG::PieceSquareTable = [
        [-50 -40 -30 -20 -20 -30 -40 -50]
        [-30 -20 -10   0   0 -10 -20 -30]
        [-30 -10  20  30  30  20 -10 -30]
        [-30 -10  30  40  40  30 -10 -30]
        [-30 -10  30  40  40  30 -10 -30]
        [-30 -10  20  30  30  20 -10 -30]
        [-30 -30   0   0   0   0 -30 -30]
        [-50 -30 -30 -30 -30 -30 -30 -50]
    ]
end

8×8 Matrix{Int64}:
 -50  -40  -30  -20  -20  -30  -40  -50
 -30  -20  -10    0    0  -10  -20  -30
 -30  -10   20   30   30   20  -10  -30
 -30  -10   30   40   40   30  -10  -30
 -30  -10   30   40   40   30  -10  -30
 -30  -10   20   30   30   20  -10  -30
 -30  -30    0    0    0    0  -30  -30
 -50  -30  -30  -30  -30  -30  -30  -50

In [24]:
if !@isdefined(PIECE_SQUARE_TABLE_BK_EG)
    const PIECE_SQUARE_TABLE_BK_EG::PieceSquareTable = reverse(PIECE_SQUARE_TABLE_WK_EG, 
    dims = 1)
end

8×8 Matrix{Int32}:
 -50  -30  -30  -30  -30  -30  -30  -50
 -30  -30    0    0    0    0  -30  -30
 -30  -10   20   30   30   20  -10  -30
 -30  -10   30   40   40   30  -10  -30
 -30  -10   30   40   40   30  -10  -30
 -30  -10   20   30   30   20  -10  -30
 -30  -20  -10    0    0  -10  -20  -30
 -50  -40  -30  -20  -20  -30  -40  -50

Die einzelnen Konstanten werden anschließend in einem Array zusammengefasst.

Es ist wichtig zu erwähnen, dass die Konstanten im Array wie folgt indexiert werden:
<br>
+ index  1 (= Chess.PIECE_WP.val)
+ index  2 (= Chess.PIECE_WN.val)
+ index  3 (= Chess.PIECE_WB.val)
+ index  4 (= Chess.PIECE_WR.val)
+ index  5 (= Chess.PIECE_WQ.val)
+ index  6 (= Chess.PIECE_WK.val)
<br>

+ index  7 (invalid)
+ index  8 (invalid)
<br>

+ index  9 (= Chess.PIECE_BP.val)
+ index 10 (= Chess.PIECE_BN.val)
+ index 11 (= Chess.PIECE_BB.val)
+ index 12 (= Chess.PIECE_BR.val)
+ index 13 (= Chess.PIECE_BQ.val)
+ index 14 (= Chess.PIECE_BK.val)
<br>

Diese Indexe, insbesondere die `invalid` Indexe sind auf die binäre Verarbeitung seitens `Chess.jl` zurückzuführen.

In [25]:
if !@isdefined(PieceSquareTables)
    const PieceSquareTables = Vector{Union{Nothing, PieceSquareTable}}
end

Vector{Union{Nothing, Matrix{Int32}}} (alias for Array{Union{Nothing, Array{Int32, 2}}, 1})

Im Schach wird zwischen der Eröffnung, dem Mittelspiel und dem Endspiel unterschieden.
<br>
Die Eröffnung bezeichnet den Beginn des Spiels. Es werden bekannte und wohldefinierte Züge verwendet, um wichtige Figuren in gute Positionen zu bewegen. Ein Beispiel ist die [Sizilianische Verteidigung](https://en.wikipedia.org/wiki/Sicilian_Defence), mit welcher auf eine `e2e4` Eröffnung reagiert wird.

Das Mittelspiel ist erreicht, sobald die Vorbereitung der Spieler abgeschlossen ist und diese anfangen den Gegner anzugreifen und Figuren zu schlagen.
<br>
In dieser Ausarbeitung werden die gleichen Werte für Eröffnung und Mittelspiel verwendet. Für das Endspiel werden neue Spielfeldbewertungen verwendet. Diese wurden ebenfalls von `Tomasz Michniewski` beschrieben. Die nachfolgenden Konstanten enthalten sowohl die Werte für das Mittel- als auch das Endspiel.

In [26]:
if !@isdefined(PIECE_SQUARE_TABLES_MIDGAME)
    const PIECE_SQUARE_TABLES_MIDGAME::PieceSquareTables = [
        PIECE_SQUARE_TABLE_WP .+ PIECE_VALUE_PAWN, 
        PIECE_SQUARE_TABLE_WN .+ PIECE_VALUE_KNIGHT, 
        PIECE_SQUARE_TABLE_WB .+ PIECE_VALUE_BISHOP, 
        PIECE_SQUARE_TABLE_WR .+ PIECE_VALUE_ROOK, 
        PIECE_SQUARE_TABLE_WQ .+ PIECE_VALUE_QUEEN, 
        PIECE_SQUARE_TABLE_WK .+ PIECE_VALUE_KING, 
        
        nothing, 
        nothing, 
        
        PIECE_SQUARE_TABLE_BP .+ PIECE_VALUE_PAWN, 
        PIECE_SQUARE_TABLE_BN .+ PIECE_VALUE_KNIGHT, 
        PIECE_SQUARE_TABLE_BB .+ PIECE_VALUE_BISHOP, 
        PIECE_SQUARE_TABLE_BR .+ PIECE_VALUE_ROOK, 
        PIECE_SQUARE_TABLE_BQ .+ PIECE_VALUE_QUEEN, 
        PIECE_SQUARE_TABLE_BK .+ PIECE_VALUE_KING, 
    ]
end

14-element Vector{Union{Nothing, Matrix{Int32}}}:
 Int32[100 100 … 100 100; 150 150 … 150 150; … ; 105 110 … 110 105; 100 100 … 100 100]
 Int32[270 280 … 280 270; 280 300 … 300 280; … ; 280 300 … 300 280; 270 280 … 280 270]
 Int32[310 320 … 320 310; 320 330 … 330 320; … ; 320 335 … 335 320; 310 320 … 320 310]
 Int32[500 500 … 500 500; 505 510 … 510 505; … ; 495 500 … 500 495; 500 500 … 500 500]
 Int32[880 890 … 890 880; 890 900 … 900 890; … ; 890 900 … 900 890; 880 890 … 890 880]
 Int32[-30 -40 … -40 -30; -30 -40 … -40 -30; … ; 20 20 … 20 20; 20 30 … 30 20]
 nothing
 nothing
 Int32[100 100 … 100 100; 105 110 … 110 105; … ; 150 150 … 150 150; 100 100 … 100 100]
 Int32[270 280 … 280 270; 280 300 … 300 280; … ; 280 300 … 300 280; 270 280 … 280 270]
 Int32[310 320 … 320 310; 320 335 … 335 320; … ; 320 330 … 330 320; 310 320 … 320 310]
 Int32[500 500 … 500 500; 495 500 … 500 495; … ; 505 510 … 510 505; 500 500 … 500 500]
 Int32[880 890 … 890 880; 890 900 … 900 890; … ; 890 900 … 900 890; 88

In [28]:
if !@isdefined(PIECE_SQUARE_TABLES_ENDGAME)
    const PIECE_SQUARE_TABLES_ENDGAME::PieceSquareTables = [
        PIECE_SQUARE_TABLE_WP    .+ PIECE_VALUE_PAWN, 
        PIECE_SQUARE_TABLE_WN    .+ PIECE_VALUE_KNIGHT, 
        PIECE_SQUARE_TABLE_WB    .+ PIECE_VALUE_BISHOP, 
        PIECE_SQUARE_TABLE_WR    .+ PIECE_VALUE_ROOK, 
        PIECE_SQUARE_TABLE_WQ    .+ PIECE_VALUE_QUEEN, 
        PIECE_SQUARE_TABLE_WK_EG .+ PIECE_VALUE_KING, 
        
        nothing, 
        nothing, 
        
        PIECE_SQUARE_TABLE_BP    .+ PIECE_VALUE_PAWN, 
        PIECE_SQUARE_TABLE_BN    .+ PIECE_VALUE_KNIGHT, 
        PIECE_SQUARE_TABLE_BB    .+ PIECE_VALUE_BISHOP, 
        PIECE_SQUARE_TABLE_BR    .+ PIECE_VALUE_ROOK, 
        PIECE_SQUARE_TABLE_BQ    .+ PIECE_VALUE_QUEEN, 
        PIECE_SQUARE_TABLE_BK_EG .+ PIECE_VALUE_KING, 
    ]
end

14-element Vector{Union{Nothing, Matrix{Int32}}}:
 Int32[100 100 … 100 100; 150 150 … 150 150; … ; 105 110 … 110 105; 100 100 … 100 100]
 Int32[270 280 … 280 270; 280 300 … 300 280; … ; 280 300 … 300 280; 270 280 … 280 270]
 Int32[310 320 … 320 310; 320 330 … 330 320; … ; 320 335 … 335 320; 310 320 … 320 310]
 Int32[500 500 … 500 500; 505 510 … 510 505; … ; 495 500 … 500 495; 500 500 … 500 500]
 Int32[880 890 … 890 880; 890 900 … 900 890; … ; 890 900 … 900 890; 880 890 … 890 880]
 Int32[-50 -40 … -40 -50; -30 -20 … -20 -30; … ; -30 -30 … -30 -30; -50 -30 … -30 -50]
 nothing
 nothing
 Int32[100 100 … 100 100; 105 110 … 110 105; … ; 150 150 … 150 150; 100 100 … 100 100]
 Int32[270 280 … 280 270; 280 300 … 300 280; … ; 280 300 … 300 280; 270 280 … 280 270]
 Int32[310 320 … 320 310; 320 335 … 335 320; … ; 320 330 … 330 320; 310 320 … 320 310]
 Int32[500 500 … 500 500; 495 500 … 500 495; … ; 505 510 … 510 505; 500 500 … 500 500]
 Int32[880 890 … 890 880; 890 900 … 900 890; … ; 890 900 … 900

***

## Funktionen

Die Funktion `pieceValueOf` bestimmt den Wert einer Spielfigur.

**Input**:
+ piece &rarr; die zu bewertende Spielfigur

**Output**:
+ der Wert der Spielfigur

In [29]:
function pieceValueOf(piece::Chess.Piece)
    @assert Chess.ptype(piece).val ∈ Chess.PAWN.val : Chess.KING.val
    PIECE_VALUES[Chess.ptype(piece).val]
end

pieceValueOf (generic function with 1 method)

Die Funktion `pieceValueAtSquareOf` bestimmt den Wert einer Spielfigur in Abhängigkeit des Spielfelds auf dem sie steht und dem aktuellen Spielzustand (Mittelspiel / Endspiel).

**Input**:
+ piece &rarr; die zu bewertende Spielfigur
+ square &rarr; das Spielfeld der Spielfigur 
+ pieceSquareTables &rarr; die aktuelle geltenden Piece Square Tables

**Output**:
+ der Wert der Spielfigur auf dem aktuellen Spielfeld

In [30]:
function pieceValueAtSquareOf(piece::Chess.Piece, square::Chess.Square, pieceSquareTables::PieceSquareTables)
    @assert square.val ∈ Chess.SQ_A8.val : Chess.SQ_H1.val
    return pieceSquareTables[piece.val][Chess.rank(square).val, Chess.file(square).val]
end

pieceValueAtSquareOf (generic function with 1 method)

Um zu überprüfen, ob in einem `SquareSet` (64-Bit Bitset, Rückgabewert aus z.B. `Chess.queens`) nur eine Figur eingetragen ist, definieren wir die Funktion `isPowerOf2`. Diese gibt `true` zurück, wenn eine Zahl `n` eine echte Potenz von 2 ist (Nur 1 Bit ist gesetzt). Sonst wird `false` zurückgegeben.

**Input**:
+ n &rarr; eine Zahl

**Output**:
+ ob die Zahl eine Potenz von zwei ist

In [31]:
function isPowerOf2(n::UInt64)::Bool
    return n & (n - 1) == 0 ? true : false
end

isPowerOf2 (generic function with 1 method)

### Feststellung des Endspiels
Wann das Endspiel erreicht wird, ist nicht genau definiert. Dementsprechend wichtig sind die Kriterien für den Beginn des Endspiels. `Tomasz Michniewski` hat diese wie folgt definiert:
<br>

+ Beide Seiten haben die Königin verloren
oder
+ jede Seite, die noch eine Königin besitzt, hat maximal eine weitere niederwertige Figur

Bei niederwertigen Figuren handelt es sich um Läufer und Springer.
<br>
Höherwertige Figuren sind Türme und die Königin.

Trifft eine und / oder beide Bedingungen ein, so wird vom `Mittelspiel` in das `Endspiel`gewechselt.
<br>

Die Funktion `isEndGame` bestimmt, ob die Kriterien für das Endspiel erfüllt werden. Ist dies der Fall so wird `true` wiedergegeben, falls nicht `false`.

**Input**:
+ board &rarr; das aktuelle Spielbrett

**Output**:
+ ob das Spielbrett im Endspiel ist
<br>
<br>


Die Funktion `isEndGameForSide` bestimmt, ob die Kriterien für das Endspiel einer Farbe erfüllt werden. Ist dies der Fall so wird `true` wiedergegeben, falls nicht `false`.

**Input**:
+ Kontext der isEndGame-Funktion
+ side &rarr; Farbe für die bestimmt werden soll, ob sie sich im Endspiel befindet

**Output**:
+ ob sich die Farbe im Endspiel befindet

In [32]:
function isEndGame(board::Chess.Board)::Bool
    
    queens = Chess.queens(board)
    pawnsAndRooks = Chess.pawns(board) ∪ Chess.rooks(board)
    minorPieces = Chess.knights(board) ∪ Chess.bishops(board)
    
    function isEndGameForSide(side::Chess.PieceColor)::Bool
        sidePieces = Chess.pieces(board, side)
        if Chess.isempty(queens ∩ sidePieces)
            return true
        end
        if isPowerOf2((minorPieces ∩ sidePieces).val)
            return true
        end
        return Chess.isempty(pawnsAndRooks ∩ sidePieces)
    end
    
    return isEndGameForSide(Chess.WHITE) & isEndGameForSide(Chess.BLACK)
end

isEndGame (generic function with 1 method)